In [76]:
import csv
import random
import numpy as np
import sklearn.impute as ski

def loadFile(filePath):
    dataset = []
    with open(filePath, 'r') as fd:
        next(fd)
        reader = csv.reader(fd, delimiter=';')
        for row in reader:
            for i in range(len(row)):
                row[i] = float(row[i])
            dataset.append(row)
    return dataset

def removeValues(dataset, percentage):
    toRemove = int(len(dataset[0])*percentage)
    print(len(dataset), percentage, toRemove)
    for row in dataset:
        for i in range(toRemove):
            feature = int(random.random() * len(row)) 
            row[feature] = np.nan
    return dataset

def handleMissingValues(dataset):
    imp = ski.SimpleImputer(missing_values=np.nan, strategy='mean')
    return imp.fit_transform(dataset)

In [78]:
red = loadFile("./winequality-red.csv")
red = removeValues(red, 0.1)
red = handleMissingValues(red)
print(red)

1599 0.1 1
[[ 7.4         0.7         0.         ...  0.56        9.4
   5.        ]
 [ 7.8         0.52762323  0.         ...  0.68        9.8
   5.        ]
 [ 7.8         0.76        0.04       ...  0.65806032  9.8
   5.        ]
 ...
 [ 6.3         0.51        0.272817   ...  0.75       11.
   6.        ]
 [ 5.9         0.645       0.12       ...  0.71       10.2
   5.        ]
 [ 6.          0.31        0.47       ...  0.66       11.
   6.        ]]
